Install required packages

In [ ]:
import sys

!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade opencv-python numpy pymodi scikit-learn --user

Import packages

In [ ]:
import numpy as np
import cv2
from IPython.display import display, Image, clear_output
import time
import modi

Define a necessary helper method

In [ ]:
def new_color():
    rgb = np.random.randint(0, 255, size=3)
    
    solid_color = np.zeros((300, 300, 3), np.uint8)
    solid_color[:] = rgb[::-1]
    _, ret = cv2.imencode('.png', solid_color)
    image = Image(data=ret)
    display(image)
    return rgb

new_color()

Intialize MODI object

In [ ]:
bundle = modi.MODI()

env = bundle.envs[0]
button = bundle.buttons[0]

Generate data set

In [ ]:
train_set = []
label_set = []

clear_output(wait=True)
new_color()
input("Place your env module on the color and press Enter when ready...")

for i in range(30):
    clear_output(wait=True)
    rgb = new_color()
    time.sleep(0.5)
    for i in range(10):
        label = rgb / 255
        data = [env.red, env.green, env.blue]
        label_set.append(label)
        train_set.append(data)
        print(f"{data} -> {label}")
        time.sleep(0.1)
    time.sleep(0.1)

clear_output()
print("Successfully generated dataset")
    
train_set = np.array(train_set)
label_set = np.array(label_set)

Check out the generated dataset and save them to ./data

In [ ]:
print(train_set.shape)
print(label_set.shape)

np.save('./data/label_set.npy', label_set, allow_pickle=True)
np.save('./data/train_set.npy', train_set, allow_pickle=True)

Training!!

In [ ]:
from sklearn.neural_network import MLPRegressor
from joblib import dump

train_set = np.load('./data/train_set.npy', allow_pickle=True)
label_set = np.load('./data/label_set.npy', allow_pickle=True)

clf = MLPRegressor(hidden_layer_sizes=(50, 50, 20, 10), activation='tanh',
                       random_state=1, verbose=True, solver='adam', validation_fraction=0.8, tol=0.00001, max_iter=500,
                   learning_rate_init=0.01, early_stopping=False, shuffle=True)

clf = clf.fit(train_set, label_set)

dump(clf, './model.joblib')

Inference!!

In [ ]:
from joblib import load
import time

clf = load('./model.joblib')

while True:
    clear_output(wait=True)
    rgb_read = np.array([env.red, env.green, env.blue])[np.newaxis, :]
    result = clf.predict(rgb_read)[0] * 255
    print(result)
    color = np.zeros((300, 300, 3), np.uint8)
    color[:] = result[::-1]
    _, ret = cv2.imencode('.png', color)
    image = Image(data=ret)
    display(image)
    if button.pressed:
        break
    time.sleep(0.5)
